In [1]:
import pandas as pd
import ast
import json
import os
import string

## 1. SETUP PATH

In [2]:
BASE_DIR = os.getcwd() 
DATA_RAW_DIR = os.path.join(BASE_DIR, "data", "raw")
DATA_PROC_DIR = os.path.join(BASE_DIR, "data", "processed")